In [1]:
!pip install -q transformers datasets accelerate evaluate rouge_score


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
from datasets import load_dataset

dataset = load_dataset("kdave/Indian_Financial_News")

# Check structure
print(dataset)
print(f"\nTotal rows: {len(dataset['train'])}")
print("\nColumns:", dataset['train'].column_names)
print("\nSample row:")
print(f"Content: {dataset['train'][0]['Content'][:200]}...")
print(f"Summary: {dataset['train'][0]['Summary']}")

README.md: 0.00B [00:00, ?B/s]

training_data_26000.csv:   0%|          | 0.00/115M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26961 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['URL', 'Content', 'Summary', 'Sentiment'],
        num_rows: 26961
    })
})

Total rows: 26961

Columns: ['URL', 'Content', 'Summary', 'Sentiment']

Sample row:
Content: US consumer spending dropped by a record in April as the COVID-19 pandemic undercut demand, buttressing expectations that the economy could contract in the second quarter at its steepest pace since th...
Summary: consumer spending plunges 13.6 percent in April. that was the biggest drop since the government started tracking series in 1959. consumer spending accounts for more than two-thirds of economic activity. economists polled by Reuters had forecast consumer spending plummeting 12.6 percent. a spokesman for the u.s. government said the data was not available.


In [3]:
import re

def clean_text(example):
    """Clean Content: remove extra spaces, newlines, special chars"""
    text = example['Content']
    text = re.sub(r'\s+', ' ', text)  # Multiple spaces to single
    text = re.sub(r'[^a-zA-Z0-9\s.,!?%-]', '', text)  # Keep basic punctuation
    example['Content'] = text.strip()

    # Also clean summary
    summary = example['Summary']
    summary = re.sub(r'\s+', ' ', summary)
    example['Summary'] = summary.strip()
    return example

# Clean dataset
dataset = dataset.map(clean_text)

# Split into train (90%) and validation (10%)
dataset = dataset['train'].train_test_split(test_size=0.1, seed=42)
print(f"Train: {len(dataset['train'])} | Validation: {len(dataset['test'])}")

Map:   0%|          | 0/26961 [00:00<?, ? examples/s]

Train: 24264 | Validation: 2697


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load Pegasus
model_name = "google/pegasus-arxiv"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Tokenization function
def preprocess_pegasus(examples):
    inputs = tokenizer(examples['Content'], max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(examples['Summary'], max_length=128, truncation=True, padding='max_length')
    inputs['labels'] = labels['input_ids']
    return inputs

# Tokenize dataset (keep URL and Sentiment for reference)
tokenized_data = dataset.map(preprocess_pegasus, batched=True, remove_columns=['Content', 'Summary'])

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-arxiv and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Map:   0%|          | 0/24264 [00:00<?, ? examples/s]

Map:   0%|          | 0/2697 [00:00<?, ? examples/s]

In [5]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
import numpy as np





# Training args
training_args = Seq2SeqTrainingArguments(
    output_dir="./pegasus-financial",
    eval_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
    
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    bf16=True,
    push_to_hub=False
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    tokenizer=tokenizer,
    
)



/tmp/ipykernel_319/1111290954.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [6]:
# Train
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss
500,3.918700
1000,1.566500
1500,1.376900
2000,1.305800


/usr/local/lib/python3.12/site-packages/transformers/modeling_utils.py:4034: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 256, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOK

TrainOutput(global_step=2277, training_loss=1.9494925638432559, metrics={'train_runtime': 470.8636, 'train_samples_per_second': 154.593, 'train_steps_per_second': 4.836, 'total_flos': 1.0516494681951437e+17, 'train_loss': 1.9494925638432559, 'epoch': 3.0})

In [7]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
    )

    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
    }

trainer.compute_metrics = compute_metrics

metrics = trainer.evaluate(
    eval_dataset=tokenized_data["test"],
    max_length=128,
    num_beams=2,   # reduce to 2 for faster eval
)

print("\nFINAL EVALUATION METRICS")
print(metrics)
predictions=trainer.predict(tokenized_data["test"])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



FINAL EVALUATION METRICS
{'eval_loss': 1.0730170011520386, 'eval_rouge1': 0.4245848343936328, 'eval_rouge2': 0.2613715549323139, 'eval_rougeL': 0.3511143578003335, 'eval_runtime': 333.2449, 'eval_samples_per_second': 8.093, 'eval_steps_per_second': 0.507, 'epoch': 3.0}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
trainer.save_model("./pegasus-financial-final")
tokenizer.save_pretrained("./pegasus-financial-final")

('./pegasus-financial-final/tokenizer_config.json',
 './pegasus-financial-final/special_tokens_map.json',
 './pegasus-financial-final/spiece.model',
 './pegasus-financial-final/added_tokens.json',
 './pegasus-financial-final/tokenizer.json')

In [9]:
SAVE_PATH = "/root/pegasus-financial-final"
# or use a mounted volume path if you have one

trainer.save_model(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

print("Model saved at:", SAVE_PATH)


Model saved at: /root/pegasus-financial-final
